In [ ]:
import HFH

##### Data Source
- 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf'
- 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf'

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


def create_index(file_path):
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    index = VectorStoreIndex.from_documents(documents)
    return index


lyft_index = create_index("./data/10k/lyft_2021.pdf")
uber_index = create_index("./data/10k/uber_2021.pdf")

In [ ]:

from llama_index.core.tools import FunctionTool


# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b
def sum(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a + b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
sum_tool = FunctionTool.from_defaults(fn=sum)

In [ ]:
from llama_index.core.agent import ReActAgent, AgentRunner
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# NOTE: lyft_index and uber_index are both SimpleVectorIndex instances
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    multiply_tool,
    sum_tool,
]

# initialize agent
# agent = ReActAgent.from_tools(query_engine_tools, verbose=True)
agent = AgentRunner.from_llm(query_engine_tools) #, verbose=True)

In [ ]:
response = agent.query("what is lyft ?")
from IPython.display import Markdown, display

display(Markdown(f"{response}"))

In [ ]:
response = agent.query("what is uber ?")
from IPython.display import Markdown, display

display(Markdown(f"{response}"))

In [ ]:
response = agent.query("what is 14/7 ?")
from IPython.display import Markdown, display

display(Markdown(f"{response}"))